In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

try:
    if __file__: exit
except NameError:
    __file__ = '.'

os.environ['KERAS_BACKEND']='tensorflow'
    
# Project
project_common_path = os.path.dirname(__file__)
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

import numpy as np

from data_utils import get_id_type_list_for_class, OUTPUT_PATH, GENERATED_DATA, to_set
from training_utils import classification_train as train, classification_validate as validate
from training_utils import exp_decay, step_decay, get_basic_imgaug_seq

from models.mini_vgg_multiclassification import get_mini_vgg

from sklearn.model_selection import KFold
from data_utils import to_set, equalized_data_classes, unique_tags, train_jpg_ids, TRAIN_ENC_CL_CSV
from data_utils import load_pretrained_model, get_label
from data_utils import DataCache

from xy_providers import tif_image_label_provider
from models.keras_metrics import binary_crossentropy_with_false_negatives


Using TensorFlow backend.


In [3]:
seed = 2017
np.random.seed(seed)

cache = DataCache(10000)  # !!! CHECK BEFORE LOAD TO FLOYD

class_index = 0
trainval_id_type_list = get_id_type_list_for_class(class_index, 'Train_tif')

from glob import glob
from data_utils import GENERATED_DATA, to_set, get_label

gen_train_files = glob(os.path.join(GENERATED_DATA, "train", "tif", "*.tif"))
gen_train_ids = [s[len(os.path.join(GENERATED_DATA, "train", "tif"))+1+len('gen_train_'):-4] for s in gen_train_files]
gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in gen_train_ids]
class_index_gen_train_ids = [id_type for id_type in gen_train_ids if np.sum(get_label(*gen_id_type_list[0], class_index=class_index)) > 0]
class_index_gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in class_index_gen_train_ids]
trainval_id_type_list = trainval_id_type_list + class_index_gen_id_type_list

class_indices = list(equalized_data_classes.keys())
class_indices.remove(class_index)

n_other_samples = int(len(trainval_id_type_list) * 1.0 / len(class_indices) / len(equalized_data_classes[class_index]))

for index in class_indices:
    id_type_list = np.array(get_id_type_list_for_class(index, 'Train_tif'))
    id_type_list = list(to_set(id_type_list) - to_set(trainval_id_type_list))
    np.random.shuffle(id_type_list)
    trainval_id_type_list.extend(id_type_list[:n_other_samples])


params = {
    'seed': seed,

    'xy_provider': tif_image_label_provider,

    'network': get_mini_vgg,
    'optimizer': 'adadelta',
    'loss': 'binary_crossentropy', # mae_with_false_negatives,
    'nb_epochs': 25,    # !!! CHECK BEFORE LOAD TO FLOYD
    'batch_size': 16,  # !!! CHECK BEFORE LOAD TO FLOYD

    'normalize_data': False,
    'normalization': '',

    'image_size': (224, 224),

    'class_index': class_index,

    # Learning rate scheduler
    'lr_kwargs': {
        'lr': 0.1,
        'a': 0.95,
        'init_epoch': 0
    },
    'lr_decay_f': exp_decay,

    'train_seq': get_basic_imgaug_seq,

    # Reduce learning rate on plateau
    'on_plateau': True,
    'on_plateau_kwargs': {
        'monitor': 'val_loss',
        'factor': 0.1,
        'patience': 3,
        'verbose': 1
    },

    'cache': cache,

#     'class_index': 0,
    'pretrained_model': 'load_best',
#     'pretrained_model': os.path.join(GENERATED_DATA, "weights", ""),

    'output_path': OUTPUT_PATH,
}

params['save_prefix_template'] = '{cnn_name}_tif_class=%i_fold={fold_index}_seed=%i' % (class_index, params['seed'])
params['input_shape'] = params['image_size'] + (7,)
params['n_classes'] = len(equalized_data_classes[class_index]) + 1


In [9]:
from data_utils import get_label
labels = np.zeros((len(trainval_id_type_list), len(equalized_data_classes[class_index])+1), dtype=np.uint8)
doubles = []
for i, it in enumerate(trainval_id_type_list):
    labels[i, :-1] = get_label(*it, class_index=class_index)
    if np.sum(labels[i, :-1]) < 1:
        labels[i, -1] = 1
    if np.sum(labels[i, :-1]) > 1:
        doubles.append(labels[i, :-1])

In [10]:
import matplotlib.pylab as plt
%matplotlib inline

In [11]:
stats = np.sum(labels, axis=0)
stats, len(trainval_id_type_list)

(array([1356, 1328,  392,  400, 1360,  836, 2784], dtype=uint64), 8356)

In [5]:
n_folds = 5
val_fold_index = 0
val_fold_indices = []  # !!! CHECK BEFORE LOAD TO FLOYD
hists = []

kf = KFold(n_splits=n_folds)
trainval_id_type_list = np.array(trainval_id_type_list)
for train_index, test_index in kf.split(trainval_id_type_list):
    train_id_type_list, val_id_type_list = trainval_id_type_list[train_index], trainval_id_type_list[test_index]

    if len(val_fold_indices) > 0:
        if val_fold_index not in val_fold_indices:
            val_fold_index += 1
            continue
                
    val_fold_index += 1
    print("\n\n ---- Validation fold index: ", val_fold_index, "/", n_folds)

    print(datetime.now(), len(train_id_type_list), len(val_id_type_list))
    assert len(to_set(train_id_type_list) & to_set(val_id_type_list)) == 0, "WTF"

    weights = None if 'pretrained_model' in params else None
    cnn = params['network'](lr=params['lr_kwargs']['lr'], weights=weights, **params)
    params['save_prefix'] = params['save_prefix_template'].format(cnn_name=cnn.name, fold_index=val_fold_index - 1)
    print("\n {} - Loaded {} model ...".format(datetime.now(), cnn.name))

    if 'pretrained_model' in params:
        load_pretrained_model(cnn, **params)
        
    print("\n {} - Start training ...".format(datetime.now()))

    
    break
    



 ---- Validation fold index:  1 / 5
2017-07-17 21:51:20.985624 6900 1725

 2017-07-17 21:51:21.431645 - Loaded mini-VGG model ...
Load best loss weights:  /Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/../output/weights/mini-VGG_tif_class=0_fold=0_seed=2017_00_val_loss=0.4504_val_precision=0.6705_val_recall=0.1857_val_f2=0.1873.h5 0.4504

 2017-07-17 21:51:23.135114 - Start training ...


In [6]:
import os
import sys
from datetime import datetime

import numpy as np
import pandas as pd

from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, Callback
import keras.backend as K
from keras import __version__ as keras_version


# Local repos:
local_repos_path = os.path.abspath(os.path.dirname(__file__))

keras_contrib_path = os.path.join(local_repos_path, "KerasContrib", "keras_contrib")
if keras_contrib_path not in sys.path:
    sys.path.append(keras_contrib_path)

imgaug_contrib_path = os.path.join(local_repos_path, "imgaug", "imgaug")
if imgaug_contrib_path not in sys.path:
    sys.path.append(imgaug_contrib_path)


from preprocessing.image.generators import ImageDataGenerator
from imgaug.imgaug import augmenters as iaa

from data_utils import GENERATED_DATA, OUTPUT_PATH, unique_tags
from metrics import score
from sklearn.metrics import mean_absolute_error
from postproc import pred_threshold

from training_utils import write_info, get_train_imgaug_seq, get_val_imgaug_seq, get_gen_flow

In [7]:
assert 'batch_size' in params, "Need batch_size"
assert 'save_prefix' in params, "Need save_prefix"
assert 'nb_epochs' in params, "Need nb_epochs"
assert 'seed' in params, "Need seed"
assert 'normalize_data' in params, "Need normalize_data"

model = cnn

samples_per_epoch = len(train_id_type_list) if 'samples_per_epoch' not in params else params['samples_per_epoch']
nb_val_samples = len(val_id_type_list) if 'nb_val_samples' not in params else params['nb_val_samples']
lr_decay_f = None if 'lr_decay_f' not in params else params['lr_decay_f']

save_prefix = params['save_prefix']
batch_size = params['batch_size']
nb_epochs = params['nb_epochs']
seed = params['seed']
normalize_data = params['normalize_data']
if normalize_data:
    assert 'normalization' in params, "Need normalization"
    normalization = params['normalization']
else:
    normalization = None

samples_per_epoch = (samples_per_epoch // batch_size + 1) * batch_size
nb_val_samples = (nb_val_samples // batch_size + 1) * batch_size

output_path = params['output_path'] if 'output_path' in params else GENERATED_DATA
weights_path = os.path.join(output_path, "weights")
if not os.path.exists(weights_path):
    os.mkdir(weights_path)

weights_filename = os.path.join(weights_path,
                                save_prefix + "_{epoch:02d}_val_loss={val_loss:.4f}")

metrics_names = list(model.metrics_names)
metrics_names.remove('loss')
for mname in metrics_names:
    weights_filename += "_val_%s={val_%s:.4f}" % (mname, mname)
weights_filename += ".h5"

model_checkpoint = ModelCheckpoint(weights_filename, monitor='val_loss',
                                   save_best_only=False, save_weights_only=True)
now = datetime.now()
info_filename = os.path.join(weights_path,
                             'training_%s_%s.info' % (save_prefix, str(now.strftime("%Y-%m-%d-%H-%M"))))

write_info(info_filename, **params)

csv_logger = CSVLogger(os.path.join(weights_path,
                                    'training_%s_%s.log' % (save_prefix, str(now.strftime("%Y-%m-%d-%H-%M")))))

In [8]:

callbacks = [model_checkpoint, csv_logger,]
if lr_decay_f is not None:
    assert 'lr_kwargs' in params and \
           isinstance(params['lr_kwargs'], dict), "Need lr_kwargs"
    _lr_decay_f = lambda e: lr_decay_f(epoch=e, **params['lr_kwargs'])
    lrate = LearningRateScheduler(_lr_decay_f)
    callbacks.append(lrate)
if 'on_plateau' in params and params['on_plateau']:
    if 'on_plateau_kwargs' in params and \
            isinstance(params['lr_kwargs'], dict):
        onplateau = ReduceLROnPlateau(**params['on_plateau_kwargs'])
    else:
        onplateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1)
    callbacks.append(onplateau)

print("\n-- Training parameters: %i, %i, %i, %i" % (batch_size, nb_epochs, samples_per_epoch, nb_val_samples))
print("\n-- Fit model")

class_weight = params.get('class_weight')
verbose = 1 if 'verbose' not in params else params['verbose']

if 'train_seq' in params:
    assert callable(params['train_seq']), "params['train_seq'] should be callable"
train_seq = get_train_imgaug_seq(seed) if 'train_seq' not in params else params['train_seq'](seed)

if 'val_seq' in params:
    assert callable(params['val_seq']), "params['val_seq'] should be callable"
val_seq = get_val_imgaug_seq(seed) if 'val_seq' not in params else params['val_seq'](seed)

train_gen, train_flow = get_gen_flow(id_type_list=train_id_type_list,
                                     imgaug_seq=train_seq,
                                     **params)

if normalize_data and normalization == '':
    params['normalization'] = 'from_save_prefix'

val_gen, val_flow = get_gen_flow(id_type_list=val_id_type_list,
                                 imgaug_seq=val_seq,
                                 **params)

np.random.seed(seed)


-- Training parameters: 16, 25, 6912, 1728

-- Fit model


In [9]:
counter = 0
for x, y in train_flow:
#     model.train_on_batch(x, y)
#     print("-- %i" % counter)
#     counter+=1
#     if counter == 10:
    break

In [12]:
y_pred = model.predict_on_batch(x)
print((y_pred > 0.35).astype(np.uint8))
print(y)
print(score(y, (y_pred > 0.35).astype(np.uint8)))

[[0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 1 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 1]
 [0 0 0 1 0 0 1]
 [0 1 0 0 0 0 0]
 [1 0 0 1 0 0 0]
 [1 0 0 1 0 0 0]
 [0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 1 1 0 0 0 0]]
[[0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]]
0.385416666667


In [26]:
for x2, y2 in val_flow:
    break

In [4]:
from metrics import score

In [28]:
y_pred = model.predict_on_batch(x2)
print((y_pred > 0.1).astype(np.uint8))
print(y2)
print(score(y2, (y_pred > 0.1).astype(np.uint8)))

[[1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 0 0 0 1 0 0]
 [1 0 0 0 0 0 0]
 [1 1 1 1 1 1 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 1 0 0 1 0 0]
 [1 0 0 0 0 0 0]]
[[0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]]
0.520833333333
